In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Heart failure

According to Wikipedia: Heart failure (HF), also known as congestive heart failure (CHF), decompensatio cordis (DC), and congestive cardiac failure (CCF), is when the heart is unable to pump sufficiently to maintain blood flow to meet the body tissue's needs for metabolism.

..
Heart failure is a common, costly, and potentially fatal condition. In 2015, it affected about 40 million people globally. Overall around 2% of adults have heart failure and in those over the age of 65, this increases to 6–10%. Rates are predicted to increase.

..

The risk of death is about 35% the first year after diagnosis, while by the second year the risk of death is less than 10% for those who remain alive. This degree of risk of death is similar to some cancers. In the United Kingdom, the disease is the reason for 5% of emergency hospital admissions

So having a model that predict accuratly heart cout save lives. A early 

In [ ]:
# save filepath to variable for easier access
heard_file_path = '../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv'
# read the data and store data in DataFrame titled melbourne_data
heard_failure_data = pd.read_csv(heard_file_path) 



In [ ]:
heard_failure_data.dtypes

check collumns with null values

In [ ]:
pd.isnull(heard_failure_data).sum()

No null values in the dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Read the data

features_all = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction',  'platelets','high_blood_pressure',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking'
       ]




features = ['age', 'anaemia',  'diabetes','creatinine_phosphokinase',
       'ejection_fraction', 'platelets','high_blood_pressure',
        'serum_sodium', 'sex','serum_creatinine', 'smoking'
       ]

numeric = ['age', 'creatinine_phosphokinase', 
           'ejection_fraction', 'platelets', 
           'serum_creatinine']
categorical = ['anaemia', 'diabetes', 'high_blood_pressure', 
               'sex', 'smoking']


# Separate target from predictors
y = heard_failure_data.DEATH_EVENT
X = heard_failure_data[features]

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# print a summary of the data in Melbourne data
heard_failure_data.describe()

299 rows could be a low number of rows to make a good model for prediction in practice. 

# Understundig Features


Before starting to build a model it seems reasonble to make a quick research of feature

## Anaemia

According to Wikipedia: Anemia (also spelled anaemia) is a decrease in the total amount of red blood cells (RBCs) or hemoglobin in the blood, or a lowered ability of the blood to carry oxygen.

## creatinine_phosphokinase

The data describes the Level of the CPK enzyme in the blood (mcg/L). I found some paper (f.e. https://pubmed.ncbi.nlm.nih.gov/22760499/) that show correlation between level of creatine kinase and heard failer.


## diabetes

From Wikipedia: Diabetes mellitus (DM), commonly known as diabetes, is a group of metabolic disorders characterized by a high blood sugar level over a prolonged period of time.

## ejection_fraction

From Wikipedia: An ejection fraction (EF) is the volumetric fraction (or portion of the total) of fluid (usually blood) ejected from a chamber (usually the heart) with each contraction (or heartbeat).

## high_blood_pressure

From Wikipedia: Hypertension (HTN or HT), also known as high blood pressure (HBP), is a long-term medical condition in which the blood pressure in the arteries is persistently elevated

## platelets

From Wikipedia: Platelets, also called thrombocytes (from Greek θρόμβος, "clot" and κύτος, "cell"), are a component of blood whose function (along with the coagulation factors) is to react to bleeding from blood vessel injury by clumping, thereby initiating a blood clot.

## serum_creatinine

From Wikipedia: Creatinine (/kriˈætɪnɪn/ or /kriˈætɪniːn/; from Greek: κρέας, romanized: kreas, lit. 'flesh') is a breakdown product of creatine phosphate from muscle and protein metabolism. It is released at a constant rate by the body (depending on muscle mass).

## serum_sodium

Sodium ions (Na+) are necessary in small amounts for some types of plants, but sodium as a nutrient is more generally needed in larger amounts by animals, due to their use of it for generation of nerve impulses and for maintenance of electrolyte balance and fluid balance. In animals, sodium ions are necessary for the aforementioned functions and for heart activity and certain metabolic functions.

## time

Follow-up period (days). I think follow-up periode is highly correlate to death, but doesnt really explaine a mortality. 


# EDA

To get a bedder understunding of feature we do some EDA. 

In [ ]:
labels = heard_failure_data['DEATH_EVENT'].value_counts().index
values = heard_failure_data['DEATH_EVENT'].value_counts()

In [ ]:
plt.subplot(title='distrubution DEATH_EVENT' )
plt.pie(values, labels=labels,autopct='%1.1f%%')
plt.show()

The classes seems to be Slighty Imbalance. I am going to adress that later.

## Explore numeric feature


In [ ]:
for feature in numeric:
    bplot=sns.boxplot(y=feature, x='DEATH_EVENT', 
                     data=heard_failure_data, 
                     width=0.5,
                     palette="colorblind")
    plt.show()

#bplot=sns.stripplot(y='serum_creatinine', 
#                    x='DEATH_EVENT',
#                   data=heard_failure_data, 
#                   jitter=True, 
#                   marker='o', 
#                   alpha=0.5,
#                   color='black')



At the first glance it seems to be a certain correlation between the features and DEATH_EVENT

## Explore categorical feature

In [ ]:
for feature in categorical:
    fig, (ax0, ax1) = plt.subplots(1, 2)
    fig.suptitle('Distribution of '+feature + '\n' 
                 + '____________________')
    
    for i in range(0,2):
        val = heard_failure_data.loc[heard_failure_data['DEATH_EVENT']==i][feature].value_counts()
        lab = heard_failure_data.loc[heard_failure_data['DEATH_EVENT']==i][feature].value_counts().index
        if i == 0:
            ax0.pie(val,labels=lab,autopct='%1.1f%%')
            ax0.set_title('DEATH_EVENT = False')
        else:
            ax1.pie(val,labels=lab,autopct='%1.1f%%')
            ax1.set_title('DEATH_EVENT = True')
        


At the first glance it seems that smoking, sex and diabets don't have any influece on a Death event. That not what we expected. 

In [ ]:
heard_failure_data['age'].describe()

So we make a age categorie each quartile.

In [ ]:
def map_aged(age):

    if age <= 51:
        return 1
    elif age <= 60:
        return 2
    elif age <= 70:
        return 3
    else:
        return 4

heard_failure_data['Age_category'] = heard_failure_data['age'].apply(map_aged)

### Exploring categorical variable with DEATH_EVENT = TRUE and age categorie

In [ ]:
colours = {1: 'C0',
           2: 'C1',
           3: 'C2',
           4: 'C3'}

for feature in categorical:
    fig, (ax0, ax1) = plt.subplots(1, 2)
    fig.suptitle('Distribution of age Categories '+feature + ' and DEATH_EVENT = TRUE' '\n' 
                 + '____________________')
    
    for i in range(0,2):
        val = heard_failure_data.loc[(heard_failure_data['DEATH_EVENT']==1) & (heard_failure_data[feature]==i) ]['Age_category'].value_counts()
        lab = heard_failure_data.loc[(heard_failure_data['DEATH_EVENT']==1) & (heard_failure_data[feature]==i) ]['Age_category'].value_counts().index
        if i == 0:
            ax0.pie(val,labels=lab,autopct='%1.1f%%',
                   colors=[colours[key] for key in lab])
            ax0.set_title(feature + ' = False')
        else:
            ax1.pie(val,labels=lab,autopct='%1.1f%%',
                   colors=[colours[key] for key in lab])
            ax1.set_title(feature +  '= True')

It seems that feature can increase death probabilty at a higher age.

### Exploring categorical variable with DEATH_EVENT = FALSE and age categorie

In [ ]:
colours = {1: 'C0',
           2: 'C1',
           3: 'C2',
           4: 'C3'}

for feature in categorical:
    fig, (ax0, ax1) = plt.subplots(1, 2)
    fig.suptitle('Distribution of age Categories '+feature + ' and DEATH_EVENT = FALSE' '\n' 
                 + '____________________')
    
    for i in range(0,2):
        val = heard_failure_data.loc[(heard_failure_data['DEATH_EVENT']==0) & (heard_failure_data[feature]==i) ]['Age_category'].value_counts()
        lab = heard_failure_data.loc[(heard_failure_data['DEATH_EVENT']==0) & (heard_failure_data[feature]==i) ]['Age_category'].value_counts().index
        if i == 0:
            ax0.pie(val,labels=lab,autopct='%1.1f%%',
                   colors=[colours[key] for key in lab])
            ax0.set_title(feature + ' = False')
        else:
            ax1.pie(val,labels=lab,autopct='%1.1f%%',
                   colors=[colours[key] for key in lab])
            ax1.set_title(feature +  '= True')

# Correlation Matrix

In [ ]:
plt.figure(figsize=(10,8))
corrMatrix = heard_failure_data.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

Time is highly correlated to a DEATH_EVENT, but we cant use it for our model, because is not usefull when a patient shows up the first team. 

# Models

Now we try to select a model to further improve it.

In [ ]:
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

models = []

w = 3

#RF = RandomForestClassifier(class_weight={0: 1, 1: w})

models.append(RandomForestClassifier(class_weight={0: 1, 1: w}))
models.append(GradientBoostingClassifier())
models.append(DecisionTreeClassifier())
models.append(LinearSVC(C=1.0))




In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.metrics import classification_report



# Bundle preprocessing and modeling code in a pipeline

for model in models:

    my_pipeline = Pipeline(steps=[
                                  ('model', model)
                                 ])

    # Preprocessing of training data, fit model 
    my_pipeline.fit(X_train, y_train
             )
    
    

    # Preprocessing of validation data, get predictions
    preds = my_pipeline.predict(X_valid)

    # Evaluate the model
    score = classification_report(y_valid, preds)
    #f1    = f1_score(y_valid, preds)
    print(str(model).split('(')[0],'\n', score)
    #print(str(model).split('(')[0],': F1:', f1)
    


We do some paramter tuning with the RandomForestClassifiert since it scored best

In [ ]:
model = RandomForestClassifier()



from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[5,50,100,150,200,250],
              'max_depth':[2,5,10,15,20,100,200],
             'criterion': ['gini', 'entropy'],
             'random_state': [0],
             'class_weight':[{0: 1, 1: 3},{0: 1, 1: 2},{0: 1, 1: 4}]
             # 'class_weight': ['balanced"']
             }

clf = GridSearchCV(model, parameters,scoring='f1')

clf.fit(X_train, y_train)
preds = clf.predict(X_valid)

        # Evaluate the model
score = classification_report(y_valid, preds)
#f1    = f1_score(y_valid, preds)
#print(str(model).split('(')[0],': Accurancy:', score)
print(score)
#print(str(model).split('(')[0],': F1:', f1)
print(clf.best_params_)
    

In [ ]:
clf.best_params_

## Permutation Importance

Test importance feature

In [ ]:
No lets take the best parameters

In [ ]:
model = RandomForestClassifier(class_weight= {0: 1, 1: 3}, 
                               criterion= 'entropy', 
                               max_depth= 2, 
                               n_estimators= 100, 
                               random_state = 0)

model.fit(X_train, y_train
             )
    
    

    # Preprocessing of validation data, get predictions
preds = model.predict(X_valid)

    # Evaluate the model
score = classification_report(y_valid, preds)
f1    = f1_score(y_valid, preds)
print(str(model).split('(')[0],': Accurancy:\n', score)

In [ ]:
perm = PermutationImportance(model, random_state=1).fit(X_valid, y_valid)
eli5.show_weights(perm, feature_names = X_valid.columns.tolist())

In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(model)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(X_valid)

# Make plot. Index of [1] is explained in text below.
shap.summary_plot(shap_values[1], X_valid)

Now let's look at the features

### ejection fraction

People with a low value have a higher change to die

### serum creatin

High value of serum creatine encrease the probiliy to die

### serum sodium

Low value of serum sodium encrease the probiliy to die

### age

older people have a higher change to die

### creatinine_phosphokinase




Smoking, gender, diabetes and high blood pressure seem to have littel importance on predicting a DEATH_EVENT which is not intuitiv. Eventuelly there are to littel amount of abservations and feautures to predict a DEATH_EVENT